# Automated Reasoning and Functional Dependencies

Full disclosure...this lecture is not terribly useful for most people but it is a good intellectual exercise. Automated reasoning is the area of computer science that is concerned with applying reasoning in the form of logic to computing systems. If given a set of assumptions and a goal, an automated reasoning system should be able to make logical inferences towards that goal automatically. 

One reason that functional dependencies are interesting is that computers can automatically make higher-level judgements about them. What does this mean? Suppose I know that $f_1$ and $f_2$ hold on a database, a computer might be able to automatically deduce that $f_3$ holds without having to actually test it out!

## Notation
Before we get started, let's define some mathematical notation. Every table $R$ can be described by $n$ rows over set of attributes (columns) $A={a_1,a_2,...,a_p}$:

| a1     | a2     | ... | ap     |
| ------ | ------ | --- | ------ |
| v1a1   | v1a2   | ... | v1ap   |
| ...    | ...    | ... | ...    |
| vna1   | vna2   | ... | vnap   |

Let $X$ be a subset of attributes $X \subseteq A$, e.g., column 1,3,4 or 3,5 or just 5. For each row $r \in R$, we can denote the projection onto the attributes $X$ as $r[X]$. $r[X]$ is a tuple of values; one representing each attribute in $X$.

A functional dependency $f_i$ is a relationship between sets of attributes and is denoted by $X \rightarrow Y$, where $X \subseteq A$ and $Y \subseteq A$. 

$f_i$ is satisfied (or it "holds" on a table $R$) if and only iff:
$$\forall r,s \in R \text{ if } r[X] = s[X] \implies r[Y] = s[Y]$$
This is a mathematical formalization of the intuitive defintion that we gave in the previous lecture.

## Armstrong's Axioms
Armstrong's axioms formalize some basic rules for reasoning about functional dependencies. They are actually quite intuitive. Think of them as information relationships: $X \rightarrow Y$ "X has enough information to uniquely determine Y".

* (Augmentation) If $X \rightarrow Y$ holds, then $XZ \rightarrow YZ$ also holds for any $Z$ ($XZ$ is short hand for $X \cup Z$. Basically, if X determines Y, then adding the same information on both sizes ($Z$) doesn't change that.

* (Transitivity) If $X \rightarrow Y$ and $Y \rightarrow Z$, then $X \rightarrow Z$. Similarly, if X determines Y and Y determines Z, clearly X determines X---follow the arrows!

* (Reflexivity) If Y is a subset of X, then it must be true that $X \rightarrow Y$. This is the trivial case where every set of attributes determine itself (or projections of itself).

*Example* We can use these basic axioms to prove more complex relationships. For example, suppose $X \rightarrow Y$ and $Y \rightarrow Z$ are true, is it true that $X \rightarrow YZ$ also holds over the same data?
* (Applying Augmentation) $YY \rightarrow YZ$ and since $Y \cup Y = Y$, this is equivalent to $Y \rightarrow YZ$
* (Applying Transitivity) $X \rightarrow Y$ and $Y \rightarrow YZ$, therefore $X \rightarrow YZ \blacksquare$


## Attribute Closure Algorithm
We can express these above rules in code in an algorithm called Attribute Closure. The set of attributes that are functionally dependent on the attribute A is called Attribute Closure of A. First, let's define a helper function that will be useful called powerset which calculates all subsets of a set:

In [16]:
from itertools import chain, combinations

#calculates the powerset of a set of numbers
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

Then, let's define a functional dependency data structure as follows:

In [22]:
ex_fds = {(1,2,3): (4,), (1,2): (5,)}
#123->4 and 12->5 (123 -> 5 is implied!)

In [25]:
def closure(attrs, fds, traversed=set()):
    '''Calculates the attribute closure of a set of attributes 
       assuming a given set of FDs is true.
    '''

    if len(attrs) <= 1:
        return set(attrs) #basecase
    
    if attrs in traversed:
        return set() #basecase
    
    #recursive case
    rtn = set()
    
    if attrs in fds:
        rtn = rtn.union(closure(fds[attrs],fds,traversed.union(set([attrs]))))
    
    for subattr in powerset(attrs):
        rtn = rtn.union(closure(subattr,fds,traversed.union(set([attrs]))))
    
    return rtn


We can run this code by doing the following:

In [26]:
closure((1,2),ex_fds)

{1, 2, 5}